In [249]:
import pandas as pd
import numpy as np
import random

n = 49 #number of records in file
s = 10 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s))

ts_df = pd.read_csv("/Users/mnksmith/Documents/Job Applications/Tot_Sprays.csv", skiprows=skip)
home_lat, home_lon = 42.360644, -71.058014
ts_df.loc[-1] = [home_lon, home_lat, 0, 0, 'Central Boston', 'None', '1 City Hall Square', 'TRUE', '10', '1 City Hall Square, Boston, MA 02201', 'Address', 0, 0, 0, 'None']  # adding a row
ts_df.index = ts_df.index + 1  # shifting index
ts_df.sort_index(inplace=True) 

In [250]:
ts_df.head(5)

,X,Y,FID,OBJECTID,Neighborho,Park_Name,Address_Te,Matched,MatchScore,MatchText,MatchType,MatchId,MatchXCoor,MatchYCoor,Match_Code
0,-71.058014,42.360644,0,0,Central Boston,None,1 City Hall Square,TRUE,10,"1 City Hall Square, Boston, MA 02201",Address,0,0.0000,0.000,None
1,-71.161191,42.355456,2,2,Allston/Brighton,Hobart Park,85 Hobart Street,TRUE,10,"85 Hobart St, Brighton, MA 02135",Address,342478,747744.0763,2954739.153,"STREET_EXACT, ADDRESS_EXACT, XY_IMPROVEMENT"
2,-71.067360,42.290490,13,14,Dorchester,Rev. Loesch Park,21 Wainwright Street,TRUE,10,"21 Wainwright St, Dorchester, MA 02124",Address,141302,773225.7874,2931179.091,"STREET_EXACT, ADDRESS_EXACT, XY_IMPROVEMENT"
3,-71.072720,42.286740,15,16,Dorchester,Roberts Park,23 Dunbar Ave,TRUE,5,"25 Dunbar Ave, Dorchester, MA 02124",Address,49880,771782.3936,2929805.227,"STREET_EXACT, ADDRESS_CLOSE, ADDRESS_NEIGHBORH..."
4,-71.089867,42.277595,24,29,Mattapan,Walker Playground,550 Norfolk Street,TRUE,10,"550 Norfolk St, Mattapan, MA 02126",Address,342029,767158.7847,2926449.910,"STREET_EXACT, ADDRESS_EXACT, XY_IMPROVEMENT"


Use the Haversine formula to calculate the distance between coordinates

In [251]:
def distance(lat1, lon1, lat2, lon2, earth_radius=6371):
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [252]:
distance(ts_df.Y[3], ts_df.X[3], ts_df.Y[0], ts_df.X[0])

8.306210152436018

Find the minimum and second-minimum distance to another node for each node

In [253]:
for i, basenode in ts_df.iterrows():
    current_min = 1000
    current_min_node = 0
    second_min = 1000
    second_min_node = 0
    total_cost = 0
    for j, node in ts_df.iterrows():
        dist = distance(basenode.Y, basenode.X, node.Y, node.X)
        if i!=j and dist<current_min:
            second_min = current_min
            second_min_node = current_min_node
            current_min = dist
            current_min_node = j
        if i!=j and dist>current_min and dist<second_min:
            second_min = dist
            second_min_node = j
    total_cost = 0.5*(current_min + second_min)
    print('(%i, %i): %.2f, (%i, %i): %.2f' % (i, current_min_node, current_min, i, second_min_node, second_min))
    ts_df.loc[i,'min_dist'] = current_min
    ts_df.loc[i,'min_node'] = current_min_node
    ts_df.loc[i,'second_dist'] = second_min
    ts_df.loc[i,'second_node'] = second_min_node
    ts_df.loc[i,'total_cost'] = total_cost

(0, 7): 2.48, (0, 5): 5.81
(1, 8): 6.79, (1, 6): 7.20
(2, 3): 0.61, (2, 10): 0.77
(3, 10): 0.44, (3, 2): 0.61
(4, 10): 1.71, (4, 3): 1.74
(5, 9): 1.16, (5, 6): 1.35
(6, 5): 1.35, (6, 9): 1.70
(7, 0): 2.48, (7, 5): 3.98
(8, 6): 5.37, (8, 4): 5.57
(9, 5): 1.16, (9, 10): 1.39
(10, 3): 0.44, (10, 2): 0.77


Setting a lower bound on the cost of a tour:

Each tot spray is located at a node, and each node is connected to the other nodes by edges.  Let's call a visit to every single node a "tour", and the distance it takes to reach all nodes the "cost" of the tour.

Cost of a tour = (1/2) * ∑ (Sum of the cost of the two edges adjacent to the node and in the tour)

We divide by two in order to avoid double counting.  So in order to set a lower bound on the cost of all possible tours, we take the two shortest edges adjacent to each node:

Cost of any tour >= (1/2) * ∑ (Sum of the cost of the two shortest edges adjacent to the node)

In [254]:
global_bound = ts_df.total_cost.sum()
print('Initial lower bound: %.2f km' % (global_bound))

Initial lower bound: 27.45 km


In [255]:
len(ts_df.index)

11

In [258]:
def ts_recursive(ts_df, current_bound, current_weight, level, current_path):
    global final_cost
    global attempt_no
    global start_level
    global start_weight
    global global_bound
    global start_path

         #If we've already reached the last node, look at the whole path
    if level==no_loc:
        current_cost = current_weight + distance(ts_df.Y[current_path[level-1]], ts_df.X[current_path[level-1]], ts_df.Y[current_path[0]], ts_df.X[current_path[0]])

        #If the total cost is smaller than the last min, update the final path
        if current_cost < final_cost:
            for k in range(0,no_loc):
                final_path[k] = current_path[k]
            final_path[no_loc] = current_path[0]
            final_cost = current_cost
            attempt_no += 1
            print('Attempt number %i: %.2f km' % (attempt_no, final_cost))
            print(final_path.values)
            level = 1
            current_weight = 0
            current_path = current_path.drop(current_path.index[1:])
            visited_nodes[:] = False
            visited_nodes[0] = True
    
    #Iterate over all nodes recursively
    for i in range(0,no_loc):
#        print(visited_nodes.values)
        #Consider a node if it's not the same one, or one we've already visited
        if (level-1 != i) and (visited_nodes[i] == False):
            temp = current_bound
            current_weight += distance(ts_df.Y[current_path[level-1]], ts_df.X[current_path[level-1]], ts_df.Y[i], ts_df.X[i])
            #Second level takes min_dist rather than second_min
            if(level==1):
                current_bound -= 0.5*(ts_df.min_dist[current_path[level-1]] + ts_df.min_dist[i])
            else:
                current_bound -= 0.5*(ts_df.second_dist[current_path[level-1]] + ts_df.min_dist[i])
            
            #Sum current_bound and current_weight to get the new lower bound for this node
            #If current lower bound < final_cost, it's worth continuing on this path
            cost_home = distance(ts_df.Y[current_path[level-1]], ts_df.X[current_path[level-1]], ts_df.Y[current_path[0]], ts_df.X[current_path[0]])
 #           print('(%i,%i)' % (level-1, i))
 #           print(current_path.values)
  #          print(current_bound + current_weight)
            if (current_bound + current_weight + cost_home < final_cost):
                current_path[level] = i
                visited_nodes[i] = True
                
                #Now iterate recursively
                ts_recursive(ts_df, current_bound, current_weight, level+1, current_path)
                
            #Otherwise we forget about this path
            #Reset current_bound, current_weight, and visited_nodes
            current_weight -= distance(ts_df.Y[current_path[level-1]], ts_df.X[current_path[level-1]], ts_df.Y[i], ts_df.X[i])
            current_bound = temp
            visited_nodes[:] = False
            for j in range(0,level):
                visited_nodes[current_path[j]] = True
                
                
                
                
                
                

In [240]:
best_bound = global_bound
best_cost = 999999

def ts_recursive(ts_df, level, old_bound, current_path):
    global best_bound
    global best_cost
    
    if(level==len(ts_df.index)):
        total_cost = 0
        for k in range(0,len(ts_df.index)):
            total_cost += distance(ts_df.Y[current_path[k]], ts_df.X[current_path[k]], ts_df.Y[k+1], ts_df.X[k+1])
        if total_cost < best_cost:
            best_cost = total_cost
    
    for i in range(0,len(ts_df.index)):
        current_bound = old_bound
        if (level-1 != i) and (visited_nodes[i] == False):                
            current_bound += distance(ts_df.Y[current_path[level-1]], ts_df.X[current_path[level-1]], ts_df.Y[i], ts_df.X[i])
            if(level==1):
                current_bound -= 0.5*(ts_df.min_dist[current_path[level-1]] + ts_df.min_dist[i])
            else:
                current_bound -= 0.5*(ts_df.second_dist[current_path[level-1]] + ts_df.min_dist[i])
            
            if(current_bound < best_bound):
                current_path[level] = i
                visited_nodes[i] = True
                
                ts_recursive(ts_df, level+1, current_bound, current_path)
                
            if(current_bound > prev_bound):
                ts_recursive(ts_df, level, old_bound, current_path)
                
            visited_nodes[:] = False
            for j in range(0,level):
                visited_nodes[current_path[j]] = True
            prev_bound = current_bound
                
            

                
                
                
                
                
                

In [259]:
final_path = pd.Series([])
final_cost = 999999
no_loc = len(ts_df.index)
visited_nodes = pd.Series([])
for j in range(0,no_loc):
    visited_nodes[j] = False
visited_nodes[0] = True
start_path = pd.Series([])
start_path[0] = 0
attempt_no = 0
start_level = 1
start_weight = 0
counter = 0

ts_recursive(ts_df, global_bound, start_weight, start_level, start_path)
print('Minimum distance: %.2f' % (final_cost))
print('Path Taken: ' + str(final_path.values))


Attempt number 1: 56.34 km
[ 0  1  2  3  4  5  6  7  8  9 10  0]
Attempt number 2: 54.76 km
[ 0  1  2  3  4  5  6  7  9 10  8  0]
Attempt number 3: 47.59 km
[ 0  1  2  3  4  5  6  8  9 10  7  0]
Attempt number 4: 47.26 km
[ 0  1  2  3  4  5  6  8 10  9  7  0]
Attempt number 5: 46.56 km
[ 0  1  2  3  4  5  9 10  8  6  7  0]
Attempt number 6: 46.22 km
[ 0  1  2  3  4  6  8 10  9  5  7  0]
Attempt number 7: 44.72 km
[ 0  1  2  3  4  8  6  5  9 10  7  0]
Attempt number 8: 44.25 km
[ 0  1  2  3  4  8  6  9 10  5  7  0]
Attempt number 9: 44.23 km
[ 0  1  2  3  4  8 10  9  6  5  7  0]
Attempt number 10: 43.51 km
[ 0  1  2  3  4 10  9  8  6  5  7  0]
Attempt number 11: 43.33 km
[ 0  1  2  3  9 10  4  8  6  5  7  0]
Attempt number 12: 42.91 km
[ 0  1  2  3 10  9  4  8  6  5  7  0]
Attempt number 13: 42.46 km
[ 0  1  4  3  2 10  9  8  6  5  7  0]
Attempt number 14: 42.27 km
[ 0  1  5  6  9  8  4 10  3  2  7  0]
Attempt number 15: 40.66 km
[ 0  1  5  9  2  3 10  4  8  6  7  0]
Attempt number 16: 

KeyboardInterrupt: 